**INSTALLATION**

In [1]:
import os
import shutil
if os.path.exists('/content/ProphecyPlus/'):
  shutil.rmtree('/content/ProphecyPlus/')


!pwd
!ls -lt

/content
total 4
drwxr-xr-x 1 root root 4096 Jan 17 14:22 sample_data


In [2]:
!git clone --recurse-submodules https://github.com/safednn-nasa/ProphecyPlus

Cloning into 'ProphecyPlus'...
remote: Enumerating objects: 5347, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 5347 (delta 74), reused 38 (delta 38), pack-reused 5248 (from 3)
Receiving objects: 100% (5347/5347), 139.54 MiB | 45.50 MiB/s, done.
Resolving deltas: 100% (2832/2832), done.


In [3]:
%cd ./ProphecyPlus
!pwd
!ls -lt dataset_models/airfoil_self_noise/

/content/ProphecyPlus
/content/ProphecyPlus
total 1668
-rw-r--r-- 1 root root 534454 Jan 22 05:31 renamed_model.onnx
-rw-r--r-- 1 root root 534434 Jan 22 05:31 model_regular_fit.onnx
-rw-r--r-- 1 root root     36 Jan 22 05:31 readme.txt
-rw-r--r-- 1 root root 562288 Jan 22 05:31 model.h5
-rw-r--r-- 1 root root    278 Jan 22 05:31 airfoil_onnx_map.csv
-rw-r--r-- 1 root root  59984 Jan 22 05:31 airfoil_self_noise.csv


In [5]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 35.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 110.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.4/33.4 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 65.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.3/229.3 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.5 MB/s eta 0:0

**LOAD DATA**

In [4]:
import numpy as np
data = np.genfromtxt("/content/ProphecyPlus/dataset_models/airfoil_self_noise/airfoil_self_noise.csv", delimiter=',')
print(data.shape)

inputstr = data[:,0:5]
outputstr = data[:,5]
#print(inputstr.shape)
#print(inputstr[0])

inputs = []
outputs = []
for indx in range(0, len(inputstr)):
  input = []
  for indx1 in range(0, len(inputstr[indx])):
    val = np.float32(inputstr[indx][indx1])
    input.append(val)
    #if (indx == 0):
    #  print(type(val), val)
    #  print(input)
  inputs.append(input)
  outputs.append(np.float32(outputstr[indx]))

print("EXAMPLE RAW INPUT AND OUTPUT.")
print(inputs[0])
print(outputs[0])

def z_score_normalize(data):
    """
    Applies z-score normalization (subtracts mean, divides by standard deviation) to the input data.

    Args:
        data (numpy.ndarray): The data to be normalized. Each row is a sample, and each column is a feature.

    Returns:
        normalized_data (numpy.ndarray): The normalized data.
        mean (numpy.ndarray): The mean of the original data (per feature).
        std (numpy.ndarray): The standard deviation of the original data (per feature).
    """
    # Convert data to NumPy array if it's a list
    data = np.asarray(data)

    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)

    # Avoid division by zero
    # Check if std is a scalar (for 1D data)
    if np.isscalar(std):
        if std == 0:
            std = 1
    else:  # For 2D data (or higher)
        std[std == 0] = 1

    normalized_data = (data - mean) / std
    return normalized_data, mean, std

(normalized_data, mean, std)= z_score_normalize(inputs)
print("EXAMPLE NORM INPUT AND OUTPUT.")
print(normalized_data[0])
(normalized_output, mean, std)= z_score_normalize(outputs)
print(normalized_output[0])


(1503, 6)
EXAMPLE RAW INPUT AND OUTPUT.
[800.0, 0.0, 0.3048, 71.3, 0.00266337]
126.201
EXAMPLE NORM INPUT AND OUTPUT.
[-0.6620218 -1.1463989  1.7992973  1.312917  -0.6448043]
0.1979379


**LOAD ONNX AND H5 MODELS**

In [5]:
import torch
import onnx
import onnxruntime

onnx_model = onnx.load('/content/ProphecyPlus/dataset_models/airfoil_self_noise/renamed_model.onnx')
onnx.checker.check_model(onnx_model)
from onnx import helper
graph = helper.printable_graph(onnx_model.graph)  # Get a printable representation of the graph
print(graph)  # Print the textual representation of the graph

graph main_graph (
  %layer_0_input_0[FLOAT, 1x5]
) initializers (
  %fc1.weight[FLOAT, 256x5]
  %fc1.bias[FLOAT, 256]
  %fc2.weight[FLOAT, 256x256]
  %fc2.bias[FLOAT, 256]
  %fc3.weight[FLOAT, 256x256]
  %fc3.bias[FLOAT, 256]
  %fc4.weight[FLOAT, 1x256]
  %fc4.bias[FLOAT, 1]
) {
  %layer_0_output_0 = Gemm[alpha = 1, beta = 1, transB = 1](%layer_0_input_0, %fc1.weight, %fc1.bias)
  %layer_1_output_0 = Relu(%layer_0_output_0)
  %layer_2_output_0 = Gemm[alpha = 1, beta = 1, transB = 1](%layer_1_output_0, %fc2.weight, %fc2.bias)
  %layer_3_output_0 = Relu(%layer_2_output_0)
  %layer_4_output_0 = Gemm[alpha = 1, beta = 1, transB = 1](%layer_3_output_0, %fc3.weight, %fc3.bias)
  %layer_5_output_0 = Relu(%layer_4_output_0)
  %layer_6_output_0 = Gemm[alpha = 1, beta = 1, transB = 1](%layer_5_output_0, %fc4.weight, %fc4.bias)
  return %layer_6_output_0
}


In [6]:
session = onnxruntime.InferenceSession("/content/ProphecyPlus/dataset_models/airfoil_self_noise/renamed_model.onnx", None)
input_name = session.get_inputs()[0].name
output_name = session.get_outputs()[0].name
#print(input_name)
#print(output_name)


results = []
for indx in range(0,len(normalized_data)):
  input_data = (normalized_data[indx]).reshape(1, -1).astype(np.float32)
  result = session.run([output_name], {input_name: input_data})
  results.append(result[0][0][0])



from sklearn.metrics import r2_score
print(np.shape(results), np.shape(outputs))
r2 = r2_score(results,normalized_output)
print("R2 Score of ONNX MODEL:", r2)

(1503,) (1503,)
R2 Score of ONNX MODEL: 0.8916182638477707


In [7]:
import numpy as np

import tensorflow as tf
import keras


print('Loading the model:')
model_h5=tf.keras.models.load_model("/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5")
print("Printing summary of the keras model:")
model_h5.summary()

# SPLIT TRAIN AND TEST SETS
len_train = int(0.80*len(normalized_data))

normalized_data_train = normalized_data[0:len_train]
normalized_data_test = normalized_data[len_train:]
normalized_output_train = normalized_output[0:len_train]
normalized_output_test = normalized_output[len_train:]

# GET PREDS
results_h5_train = model_h5.predict(normalized_data_train)
print(np.shape(results_h5_train), np.shape(normalized_output_train))
results_h5_train_flat = results_h5_train.flatten()
print(np.shape(results_h5_train_flat))
print(np.shape(normalized_output_train))
r2 = r2_score(results_h5_train_flat,normalized_output_train)
print("H5 R2 Score:", r2)

mean_abs_err = np.mean(np.abs(results_h5_train_flat - normalized_output_train))
print("H5 Mean Absolute Error:", mean_abs_err)

Loading the model:
Printing summary of the keras model:
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer_0_input_0 (InputLaye  [(None, 5)]               0         
 r)                                                              
                                                                 
 layer_0_output_0 (Dense)    (None, 256)               1536      
                                                                 
 layer_1_output_0 (Activati  (None, 256)               0         
 on)                                                             
                                                                 
 layer_2_output_0 (Dense)    (None, 256)               65792     
                                                                 
 layer_3_output_0 (Activati  (None, 256)               0         
 on)                                                             
     

**LABEL REGRESSION OUTPUTS, 0 ( Y-YIDEAL > 0.192 (MAE)) ELSE 1**

In [8]:
labels_train = []
op_vals_cor = []
op_vals_inc = []
for indx in range(0,len(normalized_data_train)):
  if (np.abs(normalized_output_train[indx] - results_h5_train_flat[indx]) >= 0.19200696):
    labels_train.append(0)
    op_vals_inc.append(results_h5_train_flat[indx])
  else:
    labels_train.append(1)
    op_vals_cor.append(results_h5_train_flat[indx])

print("TR COUNT 1:", labels_train.count(1))
print("TR COUNT 0:", labels_train.count(0))

print("MEAN CORRECT OP VAL:", np.mean(op_vals_cor))
print("MAX CORRECT OP VAL:", np.max(op_vals_cor))
print("MIN CORRECT OP VAL:", np.min(op_vals_cor))
std_dev_cor = np.std(op_vals_cor)
print("STD CORRECT OP VAL:", std_dev_cor)

print("MEAN INCORRECT OP VAL:", np.mean(op_vals_inc))
print("MAX INCORRECT OP VAL:", np.max(op_vals_inc))
print("MIN INCORRECT OP VAL:", np.min(op_vals_inc))
std_dev_inc = np.std(op_vals_inc)
print("STD CORRECT OP VAL:", std_dev_inc)



results_h5_test = model_h5.predict(normalized_data_test)
results_h5_test_flat = results_h5_test.flatten()


labels_test = []
for indx in range(0,len(normalized_data_test)):
  if (np.abs(normalized_output_test[indx] - results_h5_test_flat[indx]) >= 0.19200696):
    labels_test.append(0)
  else:
    labels_test.append(1)

print("TEST COUNT 1:", labels_test.count(1))
print("TEST COUNT 0:", labels_test.count(0))



TR COUNT 1: 788
TR COUNT 0: 414
MEAN CORRECT OP VAL: -0.08674386
MAX CORRECT OP VAL: 1.9108756
MIN CORRECT OP VAL: -3.1071632
STD CORRECT OP VAL: 0.94361866
MEAN INCORRECT OP VAL: 0.16865167
MAX INCORRECT OP VAL: 1.9617213
MIN INCORRECT OP VAL: -2.936437
STD CORRECT OP VAL: 0.93601966
10/10 [==============================] - 0s 2ms/step
TEST COUNT 1: 171
TEST COUNT 0: 130


**SENSITIVITY ANALYSIS**

In [ ]:
from maraboupy import Marabou
from maraboupy import MarabouNetworkONNX
from maraboupy.MarabouCore import *
from maraboupy.MarabouPythonic import *

## PICK AN INPUT FROM SET OF INPUTS WITH MAE < 0.192
cor_indices = np.where(np.array(labels_train) ==1)[0]
print(len(cor_indices))
print("INPUT:", normalized_data_train[cor_indices[0]])
print("IDEAL OUTPUT:", normalized_output_train[cor_indices[0]])
print("PREDICTION:", results_h5_train_flat[cor_indices[0]])

## VARY INPUT DIMENSIONS BY EPSILON
eps = 0.1
## CHECK OUTPUT IS WITHIN MAE
mae = 0.192


filename = "/content/ProphecyPlus/dataset_models/airfoil_self_noise/renamed_model.onnx"
network_a = Marabou.read_onnx(filename)
print("INPUT VARS")
invars = network_a.inputVars[0][0].flatten()
input_flat = (normalized_data_train[cor_indices[0]]).flatten()
for indx in range(0,len(invars)):
  network_a.setLowerBound(invars[indx],input_flat[indx] - eps)
  network_a.setUpperBound(invars[indx],input_flat[indx] + eps)

print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
out_max = (results_h5_train_flat[cor_indices[0]]) + mae

network_a.setLowerBound(outvars[0],out_max) # F(X) > out_max should be UNSAT
#network_a.setUpperBound(outvars,out_min)

print("F(X) > out_max should be UNSAT")
options1 = Marabou.createOptions(verbosity = 1,timeoutInSeconds=200)
sat_unsat,vals,stats = network_a.solve(options = options1)
print("sat_unsat:", sat_unsat)

if (sat_unsat == 'sat'):
  print("SAT:")
  print("vals:", vals)


if (sat_unsat == 'unsat'):
   print("RULE PROVED!!")

#################################################
print("INPUT VARS")
network_a = Marabou.read_onnx(filename)
invars = network_a.inputVars[0][0].flatten()
input_flat = (normalized_data_train[cor_indices[0]]).flatten()
for indx in range(0,len(invars)):
  network_a.setLowerBound(invars[indx],input_flat[indx] - eps)
  network_a.setUpperBound(invars[indx],input_flat[indx] + eps)

print("OUTPUT VARS")
outvars = network_a.outputVars[0].flatten()
out_min = (results_h5_train_flat[cor_indices[0]]) - mae
network_a.setUpperBound(outvars[0],out_min)
print("F(X) < out_min should be UNSAT")
options1 = Marabou.createOptions(verbosity = 1,timeoutInSeconds=200)
sat_unsat,vals,stats = network_a.solve(options = options1)
print("sat_unsat:", sat_unsat)

if (sat_unsat == 'sat'):
  print("SAT:")
  print("vals:", vals)


if (sat_unsat == 'unsat'):
   print("RULE PROVED!!")





Instructions for updating:
non-resource variables are not supported in the long term


788
INPUT: [-0.40817672 -1.1463989   1.7992973   1.312917   -0.6448043 ]
IDEAL OUTPUT: 0.3994945
PREDICTION: 0.29531392
INPUT VARS
OUTPUT VARS
F(X) > out_max should be UNSAT
unsat
sat_unsat: unsat
RULE PROVED!!
INPUT VARS
OUTPUT VARS
F(X) < out_min should be UNSAT
unsat
sat_unsat: unsat
RULE PROVED!!


In [9]:
np.save('./x_train_npy.npy',normalized_data_train)
np.save('./y_train_npy.npy',labels_train)
np.save('./x_test_npy.npy',normalized_data_test)
np.save('./y_test_npy.npy',labels_test)

In [10]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -h

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
usage: main.py analyze [-h] -tx TRAIN_FEATURES -ty TRAIN_LABELS -vx VAL_FEATURES -vy VAL_LABELS
                       [-odl] [-oal] [-sr] [-b] [-c] [-rs RANDOM_STATE] [-type TYPE]
                       [-inptype INPTYPE] [-acts ACTS] [-layer_name LAYER_NAME] [-top TOP]

options:
  -h, --help            show this help message and exit
  -tx TRAIN_FEATURES, --train_features TRAIN_FEATURES
                        Train features
  -ty TRAIN_LABELS, --train_labels TRAIN_LABELS
                        Train labels
  -vx VAL_FEATURES, --val_features VAL_FEATURES
                        Validation features
  -vy VAL_LABELS, --val_labels VAL_LABELS
                        Validation labels
  -odl, --only-dense-layers
                        Consider only dense layers
  -oal, --only-activation-laye

**Rules for behavior (0/1), from layer 5 based on (on/off) activations**

In [11]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_5_output_0' -type 3 -acts True

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
CONFIG PARAMS:
LAYER NAME: layer_5_output_0
TYPE: 3 ,INP TYPE: 0 ,ACTS: True ,Top: False
Layer Name: layer_5_output_0
Layers to be considered for fingerprinting: ['layer_5_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_5_output_0 layer
Fingerprint after layer_5_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_5_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for training set: 100% 297/297 [00:00<00:00, 1711137.76it/s]
InV 0
impure:
[[155, 97, 39, 80, 129, 7, 130, 164, 209, 160, 196, 152, 123], [0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0], -1]

Fingerprinting VAL data after layer_5_output_0 layer
Fingerprint aft

In [12]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_5_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_5_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_5_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_5_0_sup.csv",index=False)



****** RULES ********


**Rules from layer 4 based on neuron values**

In [13]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_4_output_0' -type 3

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
CONFIG PARAMS:
LAYER NAME: layer_4_output_0
TYPE: 3 ,INP TYPE: 0 ,ACTS: False ,Top: False
Layer Name: layer_4_output_0
Layers to be considered for fingerprinting: ['layer_4_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_4_output_0 layer
Fingerprint after layer_4_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_4_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for validation set: 100% 139/139 [00:00<00:00, 1259197.10it/s]
InV 0

Fingerprinting VAL data after layer_4_output_0 layer
Fingerprint after layer_4_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.


In [14]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_4_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_4_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_4_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_4_0_sup.csv",index=False)



****** RULES ********


In [16]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_3_output_0' -type 3 -acts True

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
CONFIG PARAMS:
LAYER NAME: layer_3_output_0
TYPE: 3 ,INP TYPE: 0 ,ACTS: True ,Top: False
Layer Name: layer_3_output_0
Layers to be considered for fingerprinting: ['layer_3_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_3_output_0 layer
Fingerprint after layer_3_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_3_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for training set: 100% 276/276 [00:00<00:00, 1689967.74it/s]
InV 0
impure:
[[24, 132, 31, 217, 91, 189, 104, 94], [0, 1, 0, 1, 1, 0, 0, 1], -1]

Fingerprinting VAL data after layer_3_output_0 layer
Fingerprint after layer_3_output_0. ((301, 256) inputs

In [17]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_3_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_3_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_3_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_3_0_sup.csv",index=False)



****** RULES ********


In [18]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_2_output_0' -type 3

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
CONFIG PARAMS:
LAYER NAME: layer_2_output_0
TYPE: 3 ,INP TYPE: 0 ,ACTS: False ,Top: False
Layer Name: layer_2_output_0
Layers to be considered for fingerprinting: ['layer_2_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_2_output_0 layer
Fingerprint after layer_2_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_2_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for validation set: 100% 142/142 [00:00<00:00, 1264524.77it/s]
InV 0

Fingerprinting VAL data after layer_2_output_0 layer
Fingerprint after layer_2_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.


In [19]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_2_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_2_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_2_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_2_0_sup.csv",index=False)



****** RULES ********


In [20]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_1_output_0' -type 3 -acts True

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
CONFIG PARAMS:
LAYER NAME: layer_1_output_0
TYPE: 3 ,INP TYPE: 0 ,ACTS: True ,Top: False
Layer Name: layer_1_output_0
Layers to be considered for fingerprinting: ['layer_1_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_1_output_0 layer
Fingerprint after layer_1_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_1_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for training set: 100% 287/287 [00:00<00:00, 1611466.20it/s]
InV 0
impure:
[[152, 215, 19, 122, 170, 40, 213, 173, 219, 144, 114, 140, 222, 247, 253, 137], [0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1], -1]

Fingerprinting VAL data after layer_1_out

In [21]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_1_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_1_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_1_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_1_0_sup.csv",index=False)



****** RULES ********


In [22]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/' analyze -tx ./x_train_npy.npy -ty ./y_train_npy.npy -vx ./x_test_npy.npy -vy ./y_test_npy.npy -layer 'layer_0_output_0' -type 3

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
CONFIG PARAMS:
LAYER NAME: layer_0_output_0
TYPE: 3 ,INP TYPE: 0 ,ACTS: False ,Top: False
Layer Name: layer_0_output_0
Layers to be considered for fingerprinting: ['layer_0_output_0']
Invoking Dec-tree classifier based on FEATURES

Fingerprinting TRAIN data after layer_0_output_0 layer
Fingerprint after layer_0_output_0. ((1202, 256) inputs, (1202, 256) neurons)
Inputs: (neuron signature (On/Off activations) dataset)(labels dataset)
(1202, 256) (1202,)

RULES FROM LAYER LAYER_0_OUTPUT_0 IN TERMS OF FEATURES

Obtained all paths
Processing paths for validation set: 100% 164/164 [00:00<00:00, 1161935.57it/s]
InV 0

Fingerprinting VAL data after layer_0_output_0 layer
Fingerprint after layer_0_output_0. ((301, 256) inputs, (301, 256) neurons)
PRINTING ALL RULES.


In [23]:
import pandas as pd
_output_path = "/content/ProphecyPlus/results/airfoil_self_noise/ruleset.csv"


print("****** RULES ********")
df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 1]

df_op.to_csv("./rules_0_1.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 1]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_0_1_sup.csv",index=False)


df_op = pd.read_csv(_output_path)
df_op = df_op[df_op['test_precision'] > 90.0]
df_op = df_op[df_op['label'] == 0]


df_op.to_csv("./rules_0_0.csv",index=False)

ruleset = pd.read_csv(_output_path)
ruleset = ruleset[ruleset['label'] == 0]
## TRY SORTING BY TEST PRECISION?
ruleset = ruleset.sort_values(by=['support'], ascending=False)
ruleset = ruleset.reset_index()
ruleset = ruleset[ruleset.index == 0]

ruleset.to_csv("./rules_0_0_sup.csv",index=False)



****** RULES ********


In [24]:

df_op = pd.read_csv("./rules_0_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 0 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_0_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 0 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_1_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 1 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_1_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 1 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_2_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 2 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_2_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 2 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_3_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 3 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_3_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 3 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_4_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 4 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_4_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 4 1:", train_f1,",",test_f1)

df_op = pd.read_csv("./rules_5_0.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 5 0:", train_f1,",",test_f1)



df_op = pd.read_csv("./rules_5_1.csv")
df_op = df_op[df_op.index == 0]
train_f1 = (df_op['train_f1']).values[0]
test_f1 = (df_op['test_f1']).values[0]

print("RULE 5 1:", train_f1,",",test_f1)



RULE 0 0: 9.655172413793103 , 15.602836879432624
RULE 0 1: 5.19159456118665 , 1.162790697674419
RULE 1 0: 7.88863109048724 , 8.823529411764707
RULE 1 1: 3.0000000000000004 , 2.312138728323699
RULE 2 0: 3.32541567695962 , 1.5267175572519085
RULE 2 1: 8.038976857490866 , 10.0
RULE 3 0: 1.9138755980861248 , 1.5267175572519085
RULE 3 1: 4.708798017348204 , 3.4482758620689653
RULE 4 0: 0.4819277108433735 , 1.5267175572519085
RULE 4 1: 4.950495049504951 , 1.162790697674419
RULE 5 0: 1.4388489208633093 , 1.5267175572519085
RULE 5 1: 14.806110458284373 , 1.162790697674419


**PROVE RULES.**


In [15]:
#CREATE A RULES DIRECTORY UNDER /content/ProphecyPlus/results/airfoil_self_noise/
if (os.path.exists("/content/ProphecyPlus/results/airfoil_self_noise/rules/") == False):
  os.makedirs("/content/ProphecyPlus/results/airfoil_self_noise/rules")

!cp './rules_4_1_sup.csv' '/content/ProphecyPlus/results/airfoil_self_noise/rules/ruleset.csv'

In [16]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/rules/' prove -h

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
usage: main.py prove [-h] -mp MARABOU_PATH -onx ONNX_PATH -onx_map ONNX_MAP -tx TRAIN_FEATURES -ty
                     TRAIN_LABELS [-label LAB] [-min_const MIN_CONST] [-pred PRED] [-cp CP]

options:
  -h, --help            show this help message and exit
  -mp MARABOU_PATH, --marabou_path MARABOU_PATH
                        path to Marabou folder
  -onx ONNX_PATH, --onnx_path ONNX_PATH
                        model in ONNX form
  -onx_map ONNX_MAP, --onnx_map ONNX_MAP
                        map between the layers of .h5 and .onnx models
  -tx TRAIN_FEATURES, --train_features TRAIN_FEATURES
                        Train features
  -ty TRAIN_LABELS, --train_labels TRAIN_LABELS
                        Train labels
  -label LAB, --lab LAB
                        select top rules for given l

**Post-Condition:**

**(min_val - MAE) <= F(X) <= (max_val + MAE)**

**where**

**min_val is the minimum value of the outputs of all inputs satisfying the rule.**

**max_val is the maximum value of the outputs of all inputs satisfying the rule.**


In [17]:
# CREAT CONSTS FILE
import csv
consts = []
consts.append([0,'MIN',0.192])
#consts.append([0,'MAX',0.192])

with open('/content/ProphecyPlus/results/airfoil_self_noise/rules/consts.csv', 'w') as f:
    writer = csv.writer(f)
    for indx in range(0,len(consts)):
      writer.writerow(consts[indx])

consts_read =[]
with open('/content/ProphecyPlus/results/airfoil_self_noise/rules/consts.csv', 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        print(row)
        consts_read.append(row)

print("CONSTS READ:")
print(np.shape(consts_read))



['0', 'MIN', '0.192']
CONSTS READ:
(1, 3)


In [18]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/rules/' prove -tx ./x_train_npy.npy -ty ./y_train_npy.npy -mp '/usr/local/lib/python3.11/dist-packages/maraboupy/' -onx_map '/content/ProphecyPlus/dataset_models/airfoil_self_noise/airfoil_onnx_map.csv' -onx '/content/ProphecyPlus/dataset_models/airfoil_self_noise/renamed_model.onnx' -cp '/content/ProphecyPlus/results/airfoil_self_noise/rules/consts.csv' -label 1

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
PROVE RULE for Label: 1
RULE WITH HIGHEST SUPPORT ON TRAIN DATA
LAYER, NEURONS AND SIGNATURE:
LAYER: layer_4_output_0
NEURONS: [29, 215, 181, 86, 192, 0, 70, 109, 191, 192, 55, 223, 248, 90, 97, 6, 185, 54, 254, 191, 232, 63, 87, 186]
SIGNATURE: ['<=', 0.2756440043449402, '>', -0.5447605848312378, '>', -0.05165771581232548, '<=', 0.371839240193367, '<=', 0.3212895691394806, '>', -0.12406855449080467, '<=', -0.15119200199842453, '<=', -0.1280960813164711, '>', -0.3237892985343933, '>', -0.8529289066791534, '>', -0.1978030949831009, '<=', 0.008273482322692871, '>', 0.05723067745566368, '>', -0.16278381645679474, '>', -0.44086188077926636, '<=', -0.2951846271753311, '<=', -0.02920072339475155, '<=', -0.1097075343132019, '>', -0.14150245487689972, '<=', 0.3315742313861847, '>', -0.1211231574416

**Post-Condition:**

**(min_val - MAE) <= F(X) <= (max_val + MAE)**

**where**

**min_val is the minimum value of the outputs of all inputs satisfying the rule.**

**max_val is the maximum value of the outputs of all inputs satisfying the rule.**


In [19]:
# CREAT CONSTS FILE
import csv
consts = []
consts.append([0,'MAX',0.192])

with open('/content/ProphecyPlus/results/airfoil_self_noise/rules/consts.csv', 'w') as f:
    writer = csv.writer(f)
    for indx in range(0,len(consts)):
      writer.writerow(consts[indx])

consts_read =[]
with open('/content/ProphecyPlus/results/airfoil_self_noise/rules/consts.csv', 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        print(row)
        consts_read.append(row)

print("CONSTS READ:")
print(np.shape(consts_read))



['0', 'MAX', '0.192']
CONSTS READ:
(1, 3)


In [20]:
!python -m prophecy.main -m '/content/ProphecyPlus/dataset_models/airfoil_self_noise/model.h5' -wd '/content/ProphecyPlus/results/airfoil_self_noise/rules/' prove -tx ./x_train_npy.npy -ty ./y_train_npy.npy -mp '/usr/local/lib/python3.11/dist-packages/maraboupy/' -onx_map '/content/ProphecyPlus/dataset_models/airfoil_self_noise/airfoil_onnx_map.csv' -onx '/content/ProphecyPlus/dataset_models/airfoil_self_noise/renamed_model.onnx' -cp '/content/ProphecyPlus/results/airfoil_self_noise/rules/consts.csv' -label 1

Instructions for updating:
non-resource variables are not supported in the long term
/opt/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/tools/node/bin:/tools/google-cloud-sdk/bin
PROVE RULE for Label: 1
RULE WITH HIGHEST SUPPORT ON TRAIN DATA
LAYER, NEURONS AND SIGNATURE:
LAYER: layer_4_output_0
NEURONS: [29, 215, 181, 86, 192, 0, 70, 109, 191, 192, 55, 223, 248, 90, 97, 6, 185, 54, 254, 191, 232, 63, 87, 186]
SIGNATURE: ['<=', 0.2756440043449402, '>', -0.5447605848312378, '>', -0.05165771581232548, '<=', 0.371839240193367, '<=', 0.3212895691394806, '>', -0.12406855449080467, '<=', -0.15119200199842453, '<=', -0.1280960813164711, '>', -0.3237892985343933, '>', -0.8529289066791534, '>', -0.1978030949831009, '<=', 0.008273482322692871, '>', 0.05723067745566368, '>', -0.16278381645679474, '>', -0.44086188077926636, '<=', -0.2951846271753311, '<=', -0.02920072339475155, '<=', -0.1097075343132019, '>', -0.14150245487689972, '<=', 0.3315742313861847, '>', -0.1211231574416